In [16]:
using PyPlot

INFO: Recompiling stale cache file /Users/Alex/.julia/lib/v0.4/PyPlot.ji for module PyPlot.
INFO: Recompiling stale cache file /Users/Alex/.julia/lib/v0.4/Conda.ji for module Conda.
INFO: Recompiling stale cache file /Users/Alex/.julia/lib/v0.4/PyCall.ji for module PyCall.
//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [421]:
function f(j::Int,x::Real)
    if x>0
        return x^j
    elseif x<0
        return - x^j
    else
        return 0
    end
end

function plot_func(f::Function) # This time we plot on [-1, 1]
    xs=linspace(-1,1,300)
    ys=[f(x) for x in xs]
    surf=plot(xs,ys)
end

function product_matrix(i::Int, j::Int, n::Int) 
    # This is the inner product of <x^i, x^j> when i,j < n/2 
    # or the appropriate reflection when they are >=
    # Exact expressions only. No NIntegrations
    k= Int(round(n/2))
    if i < k && j<k
        return (1 + (-1)^(i + j))/(1 + i + j) 
    elseif i >= k && j< k 
		return (1 - (-1)^((i-k) + (j)))/(1 + (i-k) + (j)) 
	elseif i < k && j >= k
		return product_matrix(j,i,n)
    else
		return (1 + (-1)^((i-k) + (j-k)))/(1 + (i-k) + (j-k)) 
    end
end

function inner_product{T<:Real}(v1::Array{T},v2::Array{T}) 
	value=0.0
	n=length(v1)
	for i in 1:n
		for j in 1:n
            if v1[i]==0 || v2[j]==0
                continue 
            else
                value += product_matrix(i-1,j-1,n)*v1[i]*v2[j]
            end
		end
	end
    return value
end

function inner_product{T<:Real}(v1::Array{T},j::Int) #we consider x^(j-1)
	value=0.0
	n=length(v1)
	for i in 1:n
        value += product_matrix(i-1,j-1,n)*v1[i]
	end
    return value
end

function gram_schmidt{T<:Real}(Q_initial::Array{Array{T,1},1}) #I think an array of arrays is easiest here
    n = length(Q_initial[1])
    k = Int(round(n/2))
    Q_final = deepcopy(Q_initial) 
    for i = 1:k    
        for j = 1:i-1 
            proj = inner_product(Q_initial[i],Q_final[j])/inner_product(Q_final[j], Q_final[j]) 
            Q_final[i] -= proj * Q_final[j]   
        end
        Q_final[i] /= sqrt(inner_product(Q_final[i], Q_final[i])) 
    end
    return Q_final
end

function legendre(k::Int)
    Q = [[i==j?1.0:0.0 for i = 1:2*(k+1)] for j = 1:(k+1)]
    Q = gram_schmidt(Q)
    return Q
end
#working 

function orthogonalize_1{T<:Real}(v::Array{T})     #modifier of v
    n=length(v)
    k= Int(round(n/2))
    legendre_polys=legendre(k-1)
    for j in 1:k
        v[j]-= inner_product(v,legendre_polys[j])/product_matrix(j-1,j-1,n) #project out
        @show legendre_polys[j]
    end
    return v
end
#working up to here

function orthogonalize_1{T<:Real}(Q_initial::Array{Array{T,1},1})
    n = length(Q_initial[1])
    k = Int(round(n/2))
    Q_final = deepcopy(Q_initial)
    legendre_polys=legendre(k-1)
    for i = 1:k
        for j in 1:k
            proj =  inner_product(Q_initial[i],legendre_polys[j])/
                        inner_product(legendre_polys[j],legendre_polys[j])
            Q_final[i] -= proj*legendre_polys[j]
        end
    end
    return Q_final
end
#working

function orthogonalize_2{T<:Real}(Q_initial::Array{Array{T,1},1})
    #k-1 vectors orth to x^k, k-2 to x^k+1 all the way to 1 vector orth to x^2k-2
    n = length(Q_initial[1])
    k = Int(round(n/2))
    Q_final = deepcopy(Q_initial)
    for i = 1:k-1
        fi=copy(Q_initial[i]) #we assume this isn't perp to x^(k+i-1) and subtract it from the next ones
        for j = i+1:k
            #fj = copy(Q_final[j])
            a= inner_product(Q_final[j],k+i)/inner_product(fi, k+i)
            Q_final[j] -= a * fi
        end
    end 
    return Q_final
end


#standard gram-schmidt process, starting at the end
function gram_schmidt_rev{T<:Real}(Q_initial::Array{Array{T,1},1}) #I think an array of arrays is easiest here
    n = length(Q_initial[1])
    k = Int(round(n/2))
    Q_final = [[0.0 for i in 1:n] for j in 1:k]
    for i = k:-1:1    #note we're going in reverse 
        fi = copy(Q_initial[i])  #we won't modify the original, so we copy
        Q_final[i] = fi          #start with Q_final = f_i, and we'll subtract projections
        for j = i+1:k #because we're going in reverse
            proj = inner_product(fi,Q_final[j])/inner_product(Q_final[j], Q_final[j]) 
            #find the projection
            Q_final[i] -= proj * Q_final[j]    #project out direction j
        end
        Q_final[i] /= sqrt(inner_product(Q_final[i], Q_final[i])) #now normalize
    end
    return Q_final
end
#seems to be working 

function DG_Basis(k::Int)
    Q = [[j==(i-k)?1.0:0.0 for i in 1:2*k] for j in 1:k]
    Q = orthogonalize_1(Q)
    Q = orthogonalize_2(Q) #check here (its close?)
    Q = gram_schmidt_rev(Q)
    return Q
end

DG_Basis (generic function with 1 method)

In [424]:
dg=DG_Basis(4)
[inner_product(dg[i],j) for j in 1:7, i in 1:4]

7x4 Array{Any,2}:
  0.0          0.0       0.0           0.0        
 -6.66593e-17  0.0       0.0           0.0        
  0.0          0.0       0.0          -1.77636e-15
 -8.1447e-17   0.0       3.55271e-15   0.0        
  0.53033      0.0       7.10543e-15   0.0        
  0.0          0.102062  0.0           0.0        
 -0.0196419    0.0       0.0175682     0.0        

In [423]:
[inner_product(dg[i],dg[j]) for i in 1:4, j in 1:4]

4x4 Array{Any,2}:
 1.0           0.0          -1.33981e-14  0.0        
 0.0           1.0           0.0          2.84217e-14
 1.69794e-15   0.0           1.0          0.0        
 0.0          -5.68434e-14   0.0          1.0        

In [387]:
dg=DG_Basis(4)

4-element Array{Array{Float64,1},1}:
 [0.0,-2.812500000000002,0.0,2.1875000000000036,1.0,0.0,0.0,0.0]   
 [-0.1875,0.0,-0.9375,0.0,0.0,1.0,0.0,0.0]                         
 [0.0,-0.31250000000000094,0.0,-0.729166666666665,0.0,0.0,1.0,0.0] 
 [0.062499999999999944,0.0,-0.9374999999999999,0.0,0.0,0.0,0.0,1.0]

In [352]:
dg=DG_Basis(5)
[inner_product(dg[i],[j==k?1.0:0.0 for k in 1:10]) for j in 1:10, i in 1:5]

10x5 Array{Float64,2}:
  0.0          -7.54746e-16   0.0           2.84217e-14   0.0        
  2.84217e-14   0.0          -3.55271e-15   0.0           0.0        
  0.0          -2.27952e-15   0.0          -1.42109e-14   0.0        
  1.77636e-14   0.0           1.77636e-15   0.0           8.88178e-15
  0.0          -3.04191e-15   0.0          -1.42109e-14   0.0        
  0.272853      0.0          -0.314711      0.0          -0.328266   
  0.0           0.051031      0.0           0.0           0.0        
 -0.0101057     0.0           0.0243376     0.0          -1.77636e-15
  0.0          -0.00765466    0.0           0.00334077    0.0        
  0.0148697     0.0          -0.0243976     0.0           0.00719059 

In [351]:
DG_Basis(3)

3-element Array{Array{Float64,1},1}:
 [0.0,-2.1213203435596544,0.0,1.414213562373097,0.0,1.1916163935587099e-14]
 [-1.8371173070873867,0.0,-9.185586535436935,0.0,9.79795897113273,0.0]     
 [0.0,-9.486832980505154,0.0,1.581138830084193,0.0,9.486832980505154]      